# libs

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
from scipy import spatial
import glob
import os

# all zipcodes

In [2]:
zipcode_df = pd.read_csv('data/processed/zipcode.csv',index_col=[0])

zipcode_df = zipcode_df.rename(columns={
    'Latitude':'latitude',
    'Longitude':'longitude',
})

# all df_pararius

In [4]:
list_of_files = glob.glob("data/temp/df_pararius_*.csv") # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
df_pararius = pd.read_csv(latest_file, index_col=[0])
df_pararius.sample()

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,interior,construction-period,plot-size,garden-surface-area,postcode,date
5669,1650.0,"3531 VL Utrecht (Laan van Nieuw-Guinea, Spinoz...",House Makassarstraat 75,The Brick Club,/house-for-rent/utrecht/8c6167fe/makassarstraat,https://casco-media-prod.global.ssl.fastly.net...,116,NaN,5,Furnished,NaN,NaN,NaN,3531VL,2021-10-11


# What is new?

In [5]:
temp1 = df_pararius['postcode'].unique()
temp2 = zipcode_df['postcode'].unique()
cep = list(set(temp1) - set(temp2))
print(len(cep))

29


In [6]:
geolocator = Nominatim(user_agent="myGeocoder")
post_code=[]
erro_code=[]

for zipcode in cep[:]:
    try:
        dict_zip = {
            'zipcode':zipcode, 
            'geolocator':geolocator.geocode(zipcode).address, 
            'latitude':geolocator.geocode(zipcode).latitude, 
            'longitude':geolocator.geocode(zipcode).longitude, 
        }
        post_code.append(dict_zip)
        print('bom - ',zipcode)

    except:
        print('erro - ',zipcode)
        erro_code.append(zipcode)

bom -  2265BT
erro -  2595BW
bom -  5961RV
erro -  1223ML
erro -  7773EE
erro -  1036LZ
erro -  5521WX
erro -  3545DC
erro -  2553DN
erro -  6515ZJ
erro -  3931XZ
erro -  2553MB
erro -  4711SJ
erro -  3551WL
bom -  1018KZ
erro -  3015GL
erro -  1441ZM
bom -  7773DG
bom -  2592EX
bom -  7642JD
bom -  1019VR
bom -  3253MS
bom -  1341CL
bom -  9254GW
bom -  1361CD
bom -  1066LH
bom -  3541HA
erro -  2595BX
bom -  3431HM


In [7]:
zipcode = pd.DataFrame(post_code)
splited = zipcode['geolocator'].str.split(",",expand=True)
zipcode['Plaats'] = splited[0]
zipcode['Provincie'] = splited[1]
zipcode = zipcode.drop(['geolocator'],1)
zipcode

<ipython-input-7-484058219571>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  zipcode = zipcode.drop(['geolocator'],1)


,zipcode,latitude,longitude,Plaats,Provincie
0,2265BT,52.081247,4.393413,Leidschendam,Leidschendam-Voorburg
1,5961RV,51.453458,6.041077,Horst,Horst aan de Maas
2,1018KZ,52.370593,4.927053,Centrum,Amsterdam
3,7773DG,52.582108,6.603512,Hardenberg,Overijssel
4,2592EX,52.085850,4.356995,Den Haag,Zuid-Holland
5,7642JD,52.351725,6.595616,Wierden,Overijssel
6,1019VR,52.368184,4.954946,Amsterdam,Noord-Holland
7,3253MS,51.824008,3.930127,Ouddorp,Goeree-Overflakkee
8,1341CL,52.358668,5.278395,Almere,Flevoland
9,9254GW,53.215116,5.947726,Hurdegaryp,Tytsjerksteradiel


# Manual Coordinates

## Not Found

In [8]:
erro_code

['2595BW',
 '1223ML',
 '7773EE',
 '1036LZ',
 '5521WX',
 '3545DC',
 '2553DN',
 '6515ZJ',
 '3931XZ',
 '2553MB',
 '4711SJ',
 '3551WL',
 '3015GL',
 '1441ZM',
 '2595BX']

## Add manually [Source](https://postcodebijadres.nl/2511EX)

In [9]:
# df_2511EX = pd.DataFrame({})
# df_2511EX['Plaats'] = ['Schiphol']
# df_2511EX['Provincie'] = ['Noord-Holland']
# df_2511EX['latitude'] = 52.2921522
# df_2511EX['longitude'] = 4.7552736
# df_2511EX['postcode'] = '1118LL'
# zipcode_df = pd.concat([zipcode_df,df_2511EX],0)
# zipcode_df.tail()

# Save list zipcodes

In [10]:
zipcode_df = pd.concat([zipcode_df,zipcode],0).drop_duplicates(subset=['postcode']).reset_index(drop=True)

<ipython-input-10-815d2116adb3>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  zipcode_df = pd.concat([zipcode_df,zipcode],0).drop_duplicates(subset=['postcode']).reset_index(drop=True)


In [11]:
zipcode_df = zipcode_df.drop(['zipcode'], axis=1)

# Add Coord into df_pararius

In [12]:
zipcode_df = zipcode_df.rename(columns={'zipcode':'postcode'})
df_pararius_coo=pd.merge(df_pararius, zipcode_df, on='postcode',how='left')

# Final info

In [13]:
df_pararius_coo = df_pararius_coo[df_pararius_coo['date'] == df_pararius_coo['date'].max()].reset_index(drop=True)

In [14]:
df_pararius_coo = df_pararius_coo.fillna(0).drop_duplicates(subset=['irl']).reset_index(drop=True)

In [15]:
df_pararius_coo.loc[df_pararius_coo['interior']==0, 'interior'] = 'Upholstered'
df_pararius_coo.loc[df_pararius_coo['status']==0, 'status'] = 'Free'

In [16]:
df_pararius_coo['price'] = df_pararius_coo['price'].astype(float)

In [17]:
# prepare link on streamlit
df_pararius_coo['url'] = 'https://www.pararius.com' + df_pararius_coo['irl']
df_pararius_coo['link'] = "<a target='_blank' href=" + (df_pararius_coo['url']).astype(str) + ">" + (df_pararius_coo['street']).astype(str) + "</a>"
df_pararius_coo['img'] = "<a href=" + df_pararius_coo['url'] + " target='blank'><img src=" + df_pararius_coo['image'] + "title='rent' width='150' height='100'/></a>"

In [18]:
try:
    stations = pd.read_csv('https://raw.githubusercontent.com/trainline-eu/stations/master/stations.csv', sep=';',low_memory=False)
    stations.to_csv('data/processed/stations.csv')
except:
    print('local')
    stations = pd.read_csv('data/processed/stations.csv')
    
stations = stations[stations['country']=='NL'][['name','latitude', 'longitude']]
stations = stations[stations['latitude'].notna()].reset_index(drop=True)


coordinates_1 = list(zip(df_pararius_coo['latitude'], df_pararius_coo['longitude']))
coordinates_2 = list(zip(stations['latitude'], stations['longitude']))
tree = spatial.KDTree(coordinates_2)


distance=[]
for i in range(len(df_pararius_coo)):
    teste = coordinates_1[i]
    distance.append(tree.query(teste)[0])

    
df_pararius_coo['train'] = distance

In [19]:
# clc_area   = ((df_pararius_coo['Living area'] - df_pararius_coo['Living area'].mean())/df_pararius_coo['Living area'].std())
# clc_room   = ((df_pararius_coo['Rooms'] - df_pararius_coo['Rooms'].mean())/df_pararius_coo['Rooms'].std())
# clc_garden = ((df_pararius_coo['garden area'] - df_pararius_coo['garden area'].mean())/df_pararius_coo['garden area'].std())
# clc_price  = ((df_pararius_coo['price'] - df_pararius_coo['price'].mean())/df_pararius_coo['price'].std())
# clc_train  = ((df_pararius_coo['train'] - df_pararius_coo['train'].mean())/df_pararius_coo['train'].std())

In [22]:
df_pararius_coo.sample()

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,interior,...,Straat,Plaats,Gemeente,Provincie,area,neighborhood,url,link,img,train
530,750.0,2595 CN Den Haag (Bezuidenhout-Oost),Apartment Juliana van Stolberglaan,Vesting Vastgoed,/apartment-for-rent/den-haag/dd8d8a65/juliana-...,https://casco-media-prod.global.ssl.fastly.net...,36,Free,2,Upholstered,...,Juliana van Stolberglaan,'s-Gravenhage,'s-Gravenhage,Zuid-Holland,Wijk 26 Bezuidenhout,Bezuidenhout-Oost,https://www.pararius.com/apartment-for-rent/de...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.005805


In [23]:
clc_area   = df_pararius_coo['surface-area']
clc_room   = df_pararius_coo['number-of-rooms']
clc_garden = df_pararius_coo['garden-surface-area']
clc_price  = df_pararius_coo['price']
clc_train  = df_pararius_coo['train'] 

In [24]:
# df_pararius_coo['deal'] = 100 * ((3*clc_area + clc_room + 2*clc_garden) / (3*clc_price + 2*clc_train))
df_pararius_coo['deal'] = ((clc_area + clc_room + clc_garden) / (clc_price + clc_train))

# Save results

In [25]:
df_pararius_coo['city'] = df_pararius_coo['irl'].str.split("/",expand=True)[2].str.replace("-", " ").str.title()

In [26]:
df_pararius_coo.to_csv('app/df_coo_pararius.csv')
zipcode_df.to_csv('data/processed/zipcode.csv')

In [28]:
df_pararius_coo

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,interior,...,Gemeente,Provincie,area,neighborhood,url,link,img,train,deal,city
0,1249.0,3012 JM Rotterdam (Cool),House Jacobusstraat 199,Woonstad Rotterdam,/house-for-rent/rotterdam/51261173/jacobusstraat,https://casco-media-prod.global.ssl.fastly.net...,86,Free,2,Shell,...,Rotterdam,Zuid-Holland,Rotterdam Centrum,Cool,https://www.pararius.com/house-for-rent/rotter...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/house-for-ren...,0.008598,0.070456,Rotterdam
1,1455.0,6213 EG Maastricht (Wolder),Apartment Médoclaan,Star Housing,/apartment-for-rent/maastricht/fe660b22/medoclaan,https://casco-media-prod.global.ssl.fastly.net...,82,Free,3,Upholstered,...,Maastricht,Limburg,0,0,https://www.pararius.com/apartment-for-rent/ma...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.045186,0.058417,Maastricht
2,900.0,1183 HB Amstelveen (Zeestratenbuurt),Studio Straat van Magelhaens 3,EarthRock Housing,/studio-for-rent/amstelveen/23176ef1/straat-va...,https://media.pararius.nl/image/PR0001659000/P...,28,Free,1,Furnished,...,Amstelveen,Noord-Holland,Wijk 00 Amstelveen,Kostverloren,https://www.pararius.com/studio-for-rent/amste...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/studio-for-re...,0.030275,0.032221,Amstelveen
3,1017.0,"1102 XA Amsterdam (Bijlmer Centrum (D,F,H))",Apartment Dalsteindreef,Urban Short Stay,/apartment-for-rent/amsterdam/93209a08/dalstei...,https://media.pararius.nl/image/PR0001659000/P...,44,Free,2,Furnished,...,Amsterdam,Noord-Holland,"Bijlmer Centrum (D,F,H)",Venserpolder Oost,https://www.pararius.com/apartment-for-rent/am...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.008709,0.045231,Amsterdam
4,2500.0,1019 DN Amsterdam (Oostelijk Havengebied),Apartment Oostelijke Handelskade 1147,Renthouse Vastgoed,/apartment-for-rent/amsterdam/9a4eab15/oosteli...,https://media.pararius.nl/image/PR0001659000/P...,105,Free,3,Furnished,...,Amsterdam,Noord-Holland,Oostelijk Havengebied,Sporenburg,https://www.pararius.com/apartment-for-rent/am...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.015671,0.043200,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5448,1500.0,1505 TJ Zaandam (Bomenbuurt),House Lijsterbesstraat,Uw Partner in Onroerende Zaken,/house-for-rent/zaandam/1b23bcc5/lijsterbesstraat,https://casco-media-prod.global.ssl.fastly.net...,86,Under option,4,Upholstered,...,Zaanstad,Noord-Holland,Wijk 11 Zaandam Zuid,Bomenbuurt,https://www.pararius.com/house-for-rent/zaanda...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/house-for-ren...,0.027698,0.059999,Zaandam
5449,3450.0,3045 LJ Rotterdam (Zestienhoven),House Overschiese Kleiweg,BenHousing,/house-for-rent/rotterdam/e5d0ae4e/overschiese...,https://casco-media-prod.global.ssl.fastly.net...,294,Under option,6,Upholstered or furnished,...,Rotterdam,Zuid-Holland,Overschie,Zestienhoven,https://www.pararius.com/house-for-rent/rotter...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/house-for-ren...,0.020183,0.086956,Rotterdam
5450,960.0,3762 GK Soest (Klaarwater),Apartment Wiardi Beckmanstraat,Regio Vastgoedmakelaars,/apartment-for-rent/soest/2e33b708/wiardi-beck...,https://casco-media-prod.global.ssl.fastly.net...,90,Under option,4,Shell,...,Soest,Utrecht,Wijk 02 Klaarwater,Klaarwater,https://www.pararius.com/apartment-for-rent/so...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.018740,0.097915,Soest
5451,1350.0,1567 LD Assendelft (Parkrijk),Apartment Wisselwachter,Uw Partner in Onroerende Zaken,/apartment-for-rent/assendelft/829433fa/wissel...,https://casco-media-prod.global.ssl.fastly.net...,76